# S3 에 저장된 모델을 SageMaker INF2 에 배포하기

이 노트북은 [MLP-KTLim/llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 의 한국어 파인 튜닝 모델을 AWS Neuron Model 로 바꾼 [Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B](https://huggingface.co/Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 사용 합니다.<br>
Amazon EC2 Inferentia2 에 Hugging Face Text Generation Inference (HF TGI) 도커 컨테이너를 이용하여, SageMaker Endpoint 에 배포 합니다.

#### 코드 참조
- [Deploy Llama 3 70B on AWS Inferentia2 with Hugging Face Optimum](https://github.com/philschmid/huggingface-inferentia2-samples/blob/main/llama3-70b/deploy-llama-3-70b-inferentia2.ipynb)



## 1. 다음과 같은 패키지를 설치 합니다.
- 처음 실행시에 아래의 주석을 제거하고 실행 하세요



In [1]:
# ! pip install huggingface_hub
# !pip install "sagemaker>=2.199.0" transformers --upgrade --quiet

## 2. SageMaker 세션 등 가져오기

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker session region: us-east-1


## 3. Hugging Face LLM Inf2 DLC (Deep Learning Container ) 가져오기

새로운 Hugging Face TGI Neuronx DLC를 사용하여 AWS Inferentia2에서 추론을 실행할 수 있습니다. sagemaker SDK의 get_huggingface_llm_image_uri 메서드를 사용하여 원하는 backend, session, region, 그리고 version에 따라 적절한 Hugging Face TGI Neuronx DLC URI를 검색할 수 있습니다. 사용 가능한 모든 버전은 여기에서 확인할 수 있습니다.
- [중요] 아래에서는 버전 version="0.0.24" 를 사용합니다. 이유는 모델의 뉴런 컴파일 버전과 일치를 시키기 위함 입니다.
    - 0.0.24 는 Neuron-cc 2.14 버전이 탑재되어 있습니다.
- 참조: [Hugging Face TGI Neuronx DLC URI](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true)



In [3]:
# from sagemaker.huggingface import get_huggingface_llm_image_uri

# # retrieve the llm image uri
# llm_image = get_huggingface_llm_image_uri(
#   "huggingface-neuronx",
#   version="0.0.23"
#   # version="0.0.24"
# )


# 신규 버전 ( 2024년 8월 22일 출시) : Neuron-CC 2.14 버전
llm_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0"
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0


## 4. HF 에서 모델 다운로드 및 S3 업로드

### 모델 다운로드
- NeuronCC-2-14 버전으로 컴파일된 모델을 다운로드 합니다.

In [6]:
import os

download_folder = "/home/ec2-user/SageMaker/models/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B"

# model_name = "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B" # Neuron-CC 2.13 버전 에서 컴파일
# 아래의 도커의 optimum-cli 를 통하여 컴파일 됨
# 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0
model_name = "Gonsoo/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B" # Neuron-CC 2.14 버전 에서 컴파일

os.makedirs(download_folder, exist_ok=True)


In [7]:
! huggingface-cli download {model_name} --local-dir {download_folder} --exclude ".cache" --exclude ".gitattributes"
! ls -al {download_folder} 

Fetching 7 files:   0%|                                   | 0/7 [00:00<?, ?it/s]Downloading 'special_tokens_map.json' to '/home/ec2-user/SageMaker/models/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B/.cache/huggingface/download/special_tokens_map.json.278b7f0f84be865c4687700ee7b3c63d89a51e18.incomplete'

special_tokens_map.json: 100%|█████████████████| 444/444 [00:00<00:00, 4.81MB/s]
Download complete. Moving file to /home/ec2-user/SageMaker/models/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B/special_tokens_map.json

tokenizer_config.json: 100%|███████████████| 51.1k/51.1k [00:00<00:00, 63.3MB/s]
Download complete. Moving file to /home/ec2-user/SageMaker/models/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B/tokenizer_config.json

675e2498ec72a928a923.neff:   0%|                    | 0.00/10.3M [00:00<?, ?B/s]

tokenizer.json:   0%|                               | 0.00/9.09M [00:00<?, ?B/s]Downloading 'compiled/2ae6fb8fd3c66e17e30f.neff' to '/home/ec2-user/SageMaker/models/AWS-NeuronCC-2-1

### 모델을 S3 에 업로드

In [8]:
desired_s3_uri = f"s3://{sagemaker_session_bucket}/inf2_model"

In [9]:
from sagemaker.s3 import S3Uploader

file_s3_path = S3Uploader.upload(local_path=download_folder, desired_s3_uri=desired_s3_uri)
print("file_s3_path: ", file_s3_path)


file_s3_path:  s3://sagemaker-us-east-1-057716757052/inf2_model


In [10]:
# ! aws s3 rm {file_s3_path} --recursive
! aws s3 ls {file_s3_path} --recursive

2024-08-30 12:49:39          1 inf2_model/.cache/huggingface/.gitignore
2024-08-30 11:31:03          0 inf2_model/.cache/huggingface/download/README.md.lock
2024-08-30 11:31:03        101 inf2_model/.cache/huggingface/download/README.md.metadata
2024-08-30 12:49:40          0 inf2_model/.cache/huggingface/download/compiled/2ae6fb8fd3c66e17e30f.neff.lock
2024-08-30 12:49:39        125 inf2_model/.cache/huggingface/download/compiled/2ae6fb8fd3c66e17e30f.neff.metadata
2024-08-30 12:49:40          0 inf2_model/.cache/huggingface/download/compiled/675e2498ec72a928a923.neff.lock
2024-08-30 12:49:40        125 inf2_model/.cache/huggingface/download/compiled/675e2498ec72a928a923.neff.metadata
2024-08-30 11:31:03          0 inf2_model/.cache/huggingface/download/compiled/bf8ef7b688d31c9ddf4e.neff.lock
2024-08-30 11:31:03        125 inf2_model/.cache/huggingface/download/compiled/bf8ef7b688d31c9ddf4e.neff.metadata
2024-08-30 11:31:03          0 inf2_model/.cache/huggingface/download/compiled/d92

## 5. 한국어 모델을 inferentia2 에 배포하기


모델을 Amazon SageMaker에 배포하기 전에 TGI Neuronx 엔드포인트 구성을 정의해야 합니다. 다음과 같은 추가 매개변수를 정의해야 합니다:

- HF_NUM_CORES: 컴파일에 사용된 Neuron 코어의 수.
- HF_BATCH_SIZE: 모델 컴파일에 사용된 배치 크기.
- HF_SEQUENCE_LENGTH: 모델 컴파일에 사용된 시퀀스 길이.
- HF_AUTO_CAST_TYPE: 모델 컴파일에 사용된 자동 캐스트 유형.

또한 기존의 TGI 매개변수도 정의해야 합니다:

- HF_MODEL_ID: "/opt/ml/model"
    - ECR 에서 추론 이미지를 다운로드 받고, 컨테이너가 생성이 됩니다. 이후에 S3 의 모델 파일은 "/opt/ml/model" 로 다운로드가 됩니다. 그래서 이 경로를 기술해야만 합니다.
- MAX_BATCH_SIZE: 모델이 처리할 수 있는 최대 배치 크기로, 컴파일에 사용된 배치 크기와 동일합니다.
- MAX_INPUT_LENGTH: 모델이 처리할 수 있는 최대 입력 길이.
- MAX_TOTAL_TOKENS: 모델이 생성할 수 있는 최대 총 토큰 수로, 컴파일에 사용된 시퀀스 길이와 동일합니다.

### 모델 배포 파라미터 설정 및 SageMaker HuggingFaceModel 생성

In [11]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.xlarge"
health_check_timeout=1800 # additional time to load the model
volume_size=64 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "/opt/ml/model",       # Path to the model in the container
    "HF_NUM_CORES": "2", # number of neuron cores
    "HF_BATCH_SIZE": "4", # batch size used to compile the model
    "HF_SEQUENCE_LENGTH": "4096", # length used to compile the model
    "HF_AUTO_CAST_TYPE": "fp16",  # dtype of the model
    "MAX_BATCH_SIZE": "4", # max batch size for the model
    "MAX_INPUT_LENGTH": "4000", # max length of input text
    "MAX_TOTAL_TOKENS": "4096", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
    "HF_TOKEN":  "None"# HF_TOKEN
}

# model_data 딕셔너리 생성
model_data = {
    'S3DataSource': {
        'S3Uri': f"{file_s3_path}/",
        'S3DataType': 'S3Prefix',
        'CompressionType': 'None'
    }
}

print("model_data: \n", model_data) 
# create HuggingFaceModel with the image uri

llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data=model_data,
  env=config
)

model_data: 
 {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-057716757052/inf2_model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


### 모델을 SageMaker Endpoint 에 배포

HuggingFaceModel을 생성한 후에는 deploy 메서드를 사용하여 Amazon SageMaker에 배포할 수 있습니다. 우리는 ml.inf2.xlarge 인스턴스 유형을 사용하여 모델을 배포할 것입니다. TGI는 자동으로 모든 Inferentia 디바이스에 걸쳐 모델을 분산하고 샤딩할 것입니다.
- 다음은 약 17분이 소요 됩니다.

In [12]:
# deactivate warning since model is compiled
llm_model._is_compiled_model = True

llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size
)

-------------------------------!

## 6. 추론

엔드포인트가 배포된 후에는 이를 통해 추론을 실행할 수 있습니다. 우리는 predictor의 predict 메서드를 사용하여 엔드포인트에서 추론을 실행할 것입니다. 생성에 영향을 미치는 다양한 매개변수로 추론을 수행할 수 있습니다. 매개변수는 페이로드의 parameters 속성에서 정의할 수 있습니다. 지원되는 매개변수는 여기에서 확인할 수 있습니다.
메시지 API를 사용하면 대화 방식으로 모델과 상호작용할 수 있습니다. 메시지의 역할과 내용을 정의할 수 있습니다. 역할은 system, assistant 또는 user일 수 있습니다. system 역할은 모델에 컨텍스트를 제공하는 데 사용되고, user 역할은 질문을 하거나 모델에 입력을 제공하는 데 사용됩니다.

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is deep learning?" }
  ]
}
```

In [15]:
# Prompt to generate
messages=[
    { "role": "system", "content": "당신은 인공지능 전문가 입니다." },
    { "role": "user", "content": "딥러닝이 무엇인지 말해 주세요?" }
  ]

# Generation arguments
parameters = {
    "model": "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 2048,
    "stop": ["<|eot_id|>"],
}

이제 테스트 해보시죠

In [17]:
%%time
chat = llm.predict({"messages" :messages, **parameters,"steam":True})

print(chat["choices"][0]["message"]["content"].strip())

딥러닝(Deep Learning)은 인공지능(AI)의 한 분야로, 인공신경망(Artificial Neural Network)을 이용하여 데이터를 학습시키는 기술입니다. 인공신경망은 인간의 뇌를 모방한 구조로, 여러 층(layer)으로 이루어진 다층 신경망을 통해 복잡한 패턴을 학습하고 예측하는 능력을 갖추고 있습니다.

딥러닝은 다음과 같은 특징을 가지고 있습니다:

1. **다층 구조**: 딥러닝 모델은 여러 층으로 이루어져 있으며, 각 층은 입력 데이터를 더 복잡한 형태로 변환하여 다음 층으로 전달합니다. 이러한 층이 많을수록 모델의 학습 능력이 향상됩니다.

2. **자동 특징 추출**: 기존의 기계 학습 알고리즘은 개발자가 수동으로 특징을 추출해야 하지만, 딥러닝은 데이터로부터 자동으로 중요한 특징을 추출할 수 있습니다.

3. **대규모 데이터 처리**: 딥러닝 모델은 대규모의 데이터를 학습시켜야 하며, 이를 위해 고성능 컴퓨팅 자원이 필요합니다.

딥러닝은 이미지 인식, 음성 인식, 자연어 처리, 자율 주행, 게임 등 다양한 분야에서 널리 사용되고 있습니다. 예를 들어, 이미지 인식 모델은 사진 속 사람, 동물, 사물 등을 인식하고 분류할 수 있으며, 음성 인식 모델은 음성 명령을 이해하고 처리할 수 있습니다. 또한, 딥러닝은 의료 분야에서도 질병 진단, 유전자 분석 등에서 중요한 역할을 하고 있습니다.<|eot_id|>
CPU times: user 4.33 ms, sys: 621 μs, total: 4.95 ms
Wall time: 16 s


## 6. 리소스 정리

SageMaker HuggingFace Model 및 엔드포인트를 삭제 합니다.


In [18]:
llm.delete_model()
llm.delete_endpoint()